# Distributing Small Model Tests

issues with distributing the libs with workbook
had to ensure libs installed

In [1]:
from pyspark.sql import SparkSession

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/spark-data/libs/xgboost4j-spark-0.90.jar,/opt/spark-data/libs/xgboost4j-0.90.jar pyspark-shell'

In [3]:
spark = SparkSession\
        .builder\
        .appName("PySpark SKLearn")\
        .master("spark://spark-master:7077")\
        .getOrCreate()

# Lib imports

lets leverage sk-dist to distribute jobs on spark

In [11]:
import pandas
import sklearn
from skdist.distribute.search import DistGridSearchCV
from sklearn.datasets import (
  load_iris
)
from xgboost import XGBClassifier

In [12]:
cv = 5
clf_scoring = "roc_auc"
reg_scoring = "neg_mean_squared_error"

In [13]:
data = load_iris()
X = data["data"]
y = data["target"]


In [14]:
grid = dict(
    learning_rate=[.05, .01],
    max_depth=[4, 6, 8],
    colsample_bytree=[.6, .8, 1.0],
    n_estimators=[100, 200, 300]
)

In [15]:
model = DistGridSearchCV(
    XGBClassifier(),
    grid, spark.sparkContext, cv=cv, scoring=clf_scoring
    )

In [16]:
model.fit(X,y)
# predictions on the driver
preds = model.predict(X)
probs = model.predict_proba(X)

# results
print("-- Grid Search --")
print("Best Score: {0}".format(model.best_score_))
print("Best colsample_bytree: {0}".format(model.best_estimator_.colsample_bytree))
print("Best learning_rate: {0}".format(model.best_estimator_.learning_rate))
print("Best max_depth: {0}".format(model.best_estimator_.max_depth))
print("Best n_estimators: {0}".format(model.best_estimator_.n_estimators))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 0.0 failed 4 times, most recent failure: Lost task 10.3 in stage 0.0 (TID 46, 172.22.0.4, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/spark/python/lib/pyspark.zip/pyspark/worker.py", line 364, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length
    return self.loads(obj)
  File "/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 580, in loads
    return pickle.loads(obj, encoding=encoding)
ModuleNotFoundError: No module named 'skdist'

	at org.apache.spark.api.python.BasePython